In [1]:
import os
import cudf as pd

In [2]:
root="/amex/"
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns

In [233]:
preds = []
for r, d, f in os.walk(root+"test"):
    if r == "/amex/test":
        for file in f:
            if ".preds" in file :
                preds.append(os.path.join(r, file))
preds[len(preds)-1]

'/amex/test/testcb.preds'

In [234]:
preddf= pd.DataFrame()
for i in range(len(preds)):
    preddf = pd.concat([preddf, pd.read_csv(preds[i], names=['customer_ID', 'Label'], header=None)], ignore_index=True, axis=0)

In [235]:
preddf = preddf.groupby(['customer_ID'], as_index=False)['Label'].mean().round().reset_index()
#pd.DataFrame(preddf)
preddf['Label'] = preddf.Label.astype("int")
#preddf.head(1)

In [236]:
valids = pd.merge(samples, preddf, on='customer_ID', how='inner')

In [237]:
print("matching", valids[valids['prediction'] == valids['Label']].count()[0], "total", valids.count()[0], "%", valids[valids['prediction'] == valids['Label']].count()[0]/valids.count()[0])

matching 242883 total 329535 % 0.7370476580636351


In [238]:
58750/79324

0.7406333518229036

In [201]:
valids.head()

,customer_ID,prediction,Label
0,01826808206e91e32dce658158d74ab275f13f0d7025f5...,0,0
1,0182815d442a54b814da25891c7681d112a7ff9e0ace02...,0,1
2,0182908946efabcd8b33f51498f131b9607380ddaa8837...,0,1
3,018297aae80afbb632da692de6b9fc91127295557f1d7a...,0,1
4,0182b3d578002e49854220564cd07dfa0a439ed357f0bb...,0,1


In [229]:
#samples = pd.read_csv(root + "sample_submission.csv")
#samples